### Python-PIL 
图形处理模块浅析

---

导入库

In [4]:
from PIL import Image
# 定义测试图片的路径
path = '/home/lantian/Pictures/01bfc95adfa3fe556dae482d4c029879'

简单示例，读取图片并旋转45°显示 (会打开系统的图片管理器显示)

In [14]:
image = Image.open(path)
# 新的图片 p
p = image.rotate(45)
p.show()

<class 'PIL.Image.Image'>


保存上述的图像到指定的目录下,保存格式是 `png`

In [16]:
p.save('/home/lantian/test.png', 'png')

原地创建缩略图并制定大小和缩略的策略

In [17]:
# Image.ANTIALIAS 等比例缩略，按照原来的维度的比例缩率得到新的缩略图，可能不是 128 * 128 大小
# 其他的缩率情况具体查看文档
p.thumbnail((128, 128), Image.ANTIALIAS)
p.show()

图片合成（融合）
1. `Image.alpha_composite(im1, im2)` : `im1, im2` 大小必须相同并且都存在 `RGBA` 四种通道才可以合成图片

2. `Image.blend(im1, im2, alpha)` : 
    * 以 `alpha` 的比例重合两个图片，计算公式如下 `out = im1 * (1 - alpha) + im2 * alpha`
    * 同样要求图片具有想吐的大小和模式

In [63]:
# new_im = Image.alpha_composite(im1, im2)
im1 = Image.open('/home/lantian/test.png')
im2 = Image.open('/home/lantian/test2.png')
out = Image.blend(im1, im2, 0.1)
out.show()

图像计算
   * `im_new = Image.eval(im, function)`
   * 逐个像素应用函数 `function` 处理并返回新的图片

In [23]:
# 每一个像素加大`2`倍显示
im_new = Image.eval(im1, lambda i : i * 2)
im_new.show()

创建图像
1. `Image.new(mode, size, color=0)`
   * `mode` : 
       * L : 灰度图像
       * RGB : 三色通道图像
       * RGBA : 三色和透明度图像
   * `size` : `(width, height)` 元组表示图片的大小
   * `color` : 和 `mode` 对应的像素初始化元组,默认是黑色（0）
       * 灰度图像 : `255` - 白色
       * RGB : (20, 200, 45) - 三色通道
       * RGBA : (20, 200, 45, 255) - 四个通道的默认
       
2. `Image.fromarray(obj, mode=None)`
    1. 使用数组对象创建图像，可以使用 `numpy.ndarray` , 这一点对 `tensorflow` 以及 `numpy` 非常友好
    2. `mode` : 决定使用的图像的类型，灰度图像，RGB,RGBA都可以

In [68]:
# 创建灰度图像并显示,颜色是白色
im = Image.new('L', (50, 50))
# im.show()

# 创建RGB图像并显示
im = Image.new('RGB', (50, 50), (20, 200, 45))
# im.show()

# 创建RGBA图像并显示
im = Image.new('RGBA', (50, 50), (20, 200, 45, 0))
# im.show()

# 从数组创建
array = np.random.randint(0, 256, size=(50, 50))
# 创建灰度图像
im = Image.fromarray(array, mode = 'L')
im.show()

获取图像像素数据  
`Image.getdata(band=None)`
   * 数据被展平，没有了 `height` 和 `width` 只存在序列和通道
   * `mode` 可以制定被返回的通道,比如 0 只返回了通道中的红色通道数据,一次类推
   * 返回迭代器

`Image.getpixel((x, y))`  
`Image.tobytes()` : 这个函数在 `tensorflow` 创建图像的 `TFRecord` 文件的时候经常使用

In [51]:
print(list(im.getdata())[:5])
# 只获取红色通道的像素数据
print(list(im.getdata(0))[:5])

[(25, 0, 0), (0, 0, 0), (0, 0, 133), (0, 0, 0), (0, 0, 0)]
[25, 0, 0, 0, 0]


In [83]:
# 获取像素点
im.getpixel((0, 0))

15

图像的缩放，和尺寸调整
   **非常重要的功能，对于 `Tensorflow` 和其他的图像处理任务来说**

`Image.resize(size, resample = 0, box=None)`
   * `size` : `(width, height)`, 对图片的制定大小的缩放
   * `resample` : 缩放的模式，采用的模式

In [54]:
im2.resize((128, 128)).show()

图像的灰度化
1. 手动灰度化
2. 参数方法 ： `Image.convert(mode=None,matrix=None,dither=None,palette=0,color=256)`
   * `mode` : L, RGB, RGBA

In [78]:
# 手动灰度化
im_2 = im2.resize((128, 128))
im_data_2 = list(im_2.getdata())

pause = []

for i in im_data_2:
    # 均值灰度化
    pause.append([(i[0] + i[1] + i[2]) / 3])

pause = np.array(pause).reshape(128, 128)
im_2 = Image.fromarray(pause)
im_2.show()

# convert 转换成灰度图
im22 = im2.resize((128, 128))
im_2 = im22.convert('L')
im_2.show()

图像基本信息获取
1. 图像的大小 ： `img.size`
2. 图像的`mode` : `img.mode`
3. 图像的格式 : `img.format`

In [80]:
im2.size, im2.mode, im2.format

((1920, 1080), 'RGB', 'JPEG')